In [4]:
import io, json, codecs, os, fix

In [13]:
def redif2jsonld(jsonld_src,journal_name,issn,file_name):
    global articles; articles=[]
    global article; article={}
    global authors; authors=[]
    global author; author=''
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(author)>0: authors.append(author); author=''
                article['e:author']=authors
                authors=[]
                if fix.validate(article)==True: articles.append(article)
                else: e=e+1
            article={}
            article['e:author']=[]
            article['e:keyword']=[]
            article['e:journal']=journal_name
            article['e:issn']=issn
            article['e:title']=''
            article['e:year']=0
        elif key=='author-name':
            if len(author)>0: authors.append(author)
            author=''
            if ',' in j: j=' '.join(reversed([part.strip() for part in j.split(',')]))
            author=j
        elif key=='title':article['e:title']=j
        elif key=='year': 
            try: article['e:year']=int(j) 
            except: pass
        elif key=='pages':article['e:pages']=j
        elif key=='volume':article['e:volume']=j
        elif key=='issue':article['e:issue']=j
        elif key=='file-url':article['e:url']=j;article['e:doi']=j.replace('http://dx.doi.org/','').replace('http://hdl.handle.net/','')
        elif key=='doi':article['e:doi']=j
        elif key=='abstract':article['e:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='': article['e:keyword'].append(fix.fix_keyword(keyword))

    if len(article)>0:
        if len(author)>0: authors.append(author); author=''
        article['e:author']=authors; authors=[]
        articles.append(article)   
        
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_src+issn+'_'+file_name.replace('.redif','').replace('.rdf','')+'.jsonld'
    #if not os.path.exists(jsonld_file):
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    return (jsonld_file)

In [14]:
redif_folder='redif/'
jsonld_src='jsonld_src/'

    
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
print len(journals)
    
for journal in journals:
    source=journal['jname']
    #if source not in  ['ormnsc','ormsom']: continue
    journal_name=journal['journal']
    data=journal['data']
    issn=journal['issn']
    if issn !='':
        for type_pub in data:   
            file_type=type_pub['type']
            publisher=type_pub['publisher']
            if file_type=='redif' or file_type=='parse':
                if file_type=='parse': publisher='src'
                print publisher,source,journal_name,
                i=0
                e=0
                for file_name in os.listdir(redif_folder+publisher+'/'+source):
                    if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                        redif_file=publisher+'/'+source+'/'+file_name
                        content = []
                        articles=[]
                        with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                            for line in file:
                                line = line.strip()
                                content.append(line)
                        jsonld_file=redif2jsonld(jsonld_src,journal_name,issn,file_name)
                    i=i+1
                print ', INVALID=',e,'files=',i

40
src adscqt Administrative Science Quarterly , INVALID= 260 files= 1
src jrnmng Journal of Management , INVALID= 17 files= 1
bla jomstd Journal of Management Studies , INVALID= 1 files= 63
eee respol Research Policy , INVALID= 9 files= 1
bla stratm Strategic Management Journal , INVALID= 0 files= 34
oup jconrs Journal of Consumer Research , INVALID= 1082 files= 180
inm ormksc Marketing Science , INVALID= 6 files= 6
src jrnmkt Journal of Marketing , INVALID= 0 files= 1
src jrmktr Journal of Marketing Research , INVALID= 0 files= 1
inm orisre Information Systems Research , INVALID= 1 files= 1
src jrnmis Journal of Management Information Systems , INVALID= 1 files= 1
inm ormnsc Management Science , INVALID= 442 files= 6
inm ormsom Manufacturing & Service Operations Management , INVALID= 4 files= 7
inm oropre Operations Research , INVALID= 40 files= 3
src popmgt Production and Operations Management , INVALID= 1 files= 1


In [16]:
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)
journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
issns=[journal['issn'] for journal in journals if journal['issn'] != '']
    
jsonld_src_folder='jsonld_src/'
jsonld_folder='jsonld/'

for issn in issns:
    with open(jsonld_folder+'doi_'+issn+'.jsonld') as data_file:    
        doi_json = json.load(data_file)
        dois=[art['e:doi'] for art in doi_json]; 
        titles=[art['e:title'] for art in doi_json]; 
    print issn , len(dois),
    new_articles=[]
    for issn_json in os.listdir(jsonld_src_folder):
        if issn_json[0:9]==issn:
            with open(jsonld_src_folder+issn_json) as data_file:    
                src_json = json.load(data_file)
            for src_article in src_json:
                new_article={}
                if 'e:doi' in src_article:
                    src_doi=src_article['e:doi']
                    src_title=src_article['e:title']
                    if src_doi in dois or src_title in titles:
                        if src_doi in dois:
                            doi_article=[art for art in doi_json if art['e:doi']==src_doi][0]
                        if src_title in titles:
                            doi_article=[art for art in doi_json if art['e:title']==src_title][0]
                        for item in ['e:year','e:volume','e:issue','e:keyword','e:abstract']:
                            if item in src_article and item not in doi_article:
                                new_article[item]=src_article[item]
                        new_article['@id']=doi_article['@id']

                    else:
                        if issn=='0742-1222':
                            if src_article['e:year']>=1990:
                                for item in src_article:
                                    new_article[item]=src_article[item]
                                new_article['@id']=new_article.pop('e:url')
                            
                if len(new_article)>0:
                    new_articles.append(new_article)
            
    jd=unicode(json.dumps(new_articles, ensure_ascii=False))
    jsonld_file=jsonld_folder+'src_'+issn+'.jsonld'
    #if not os.path.exists(jsonld_file):
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    print len(new_articles)

0001-8392 1368 653
0149-2063 2086 1649
0022-2380 1481 916
0048-7333 2730 2062
0143-2095 2361 313
0093-5301 1575 921
0732-2399 1234 1213
0022-2429 1247 740
0022-2437 1645 961
1047-7047 864 807
0742-1222 931 1184
0025-1909 4138 3789
1523-4614 631 609
0030-364X 2632 2598
1059-1478 1478 886
